In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market

In [3]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
Market.open("BN", "BTCBUSD")

In [5]:
rbot.Market.download(10)


file to download: 'BTCBUSD-trades-2022-12-06.zip'
will be located under: '"/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/.tmpv37EAW/BTCBUSD-trades-2022-12-06.zip"'
file to download: 'BTCBUSD-trades-2022-12-07.zip'
will be located under: '"/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/.tmpupsF7V/BTCBUSD-trades-2022-12-07.zip"'


In [6]:
rbot.Market.cache_data()

# rbot.init_debug_log()

back_tester = BackTester("BN", "BTCBUSD", False)


In [7]:

print("start")
r = back_tester.run(Agent())
print("end")

print(r)



start
end
[update_time: 1664992800653000, order_id: "0000-0001", order_sub_id: 0, order_side: Sell, post_only: true, create_time: 1664992799896000, status: OpenPosition, open_price: 20168.24, open_home_size: 0.01, open_foreign_size: 201.68240000000003, close_price: 0.0, close_home_size: 0.0, close_foreign_size: 0.0, order_price: 20168.24, order_home_size: 0.01, order_foreign_size: 201.68240000000003, profit: 0.0, fee: 0.020168240000000004, total_profit: -0.020168240000000004, position_change: -0.01, message: "Open Short", update_time: 1665151201114000, order_id: "0000-0002", order_sub_id: 0, order_side: Buy, post_only: true, create_time: 1665151199995000, status: ClosePosition, open_price: 20168.24, open_home_size: 0.01, open_foreign_size: 201.68240000000003, close_price: 19605.1, close_home_size: 0.01, close_foreign_size: 196.051, order_price: 19605.1, order_home_size: 0.01, order_foreign_size: 196.051, profit: 5.631400000000042, fee: 0.0196051, total_profit: 5.611794900000042, positi

In [8]:
import rbot
df = rbot.result_to_df(r)

In [9]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-10-05 17:59:59.896000+00:00,2022-10-05 18:00:00.653000+00:00,0000-0001,0,Sell,True,Open,20168.24,0.01,201.6824,0.00,...,20168.24,0.01,201.6824,0.0000,0.020168,-0.020168,-0.01,Open Short,-0.020168,-0.01
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,0,Buy,True,Close,20168.24,0.01,201.6824,19605.10,...,19605.10,0.01,196.0510,5.6314,0.019605,5.611795,0.01,doten Long,5.591627,0.00
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,1,Buy,True,Open,19605.10,0.01,196.0510,0.00,...,19605.10,0.01,196.0510,0.0000,0.019605,-0.019605,0.01,doten Long,5.572022,0.01
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,0,Sell,True,Close,19605.10,0.01,196.0510,19474.00,...,19474.00,0.01,194.7400,-1.3110,0.019474,-1.330474,-0.01,Doten Short,4.241548,0.00
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,1,Sell,True,Open,19474.00,0.01,194.7400,0.00,...,19474.00,0.01,194.7400,0.0000,0.019474,-0.019474,-0.01,Doten Short,4.222074,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0050,1,Buy,True,Open,16949.21,0.01,169.4921,0.00,...,16949.21,0.01,169.4921,0.0000,0.016949,-0.016949,0.01,doten Long,-45.023869,0.01
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,0,Sell,True,Close,16949.21,0.01,169.4921,16968.66,...,16968.66,0.01,169.6866,0.1945,0.016969,0.177531,-0.01,Doten Short,-44.846338,0.00
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,1,Sell,True,Open,16968.66,0.01,169.6866,0.00,...,16968.66,0.01,169.6866,0.0000,0.016969,-0.016969,-0.01,Doten Short,-44.863306,-0.01


In [12]:
df = result_to_df(r)

In [14]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_home_size,open_foreign_size,close_price,...,order_price,order_home_size,order_foreign_size,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-10-05 17:59:59.896000+00:00,2022-10-05 18:00:00.653000+00:00,0000-0001,0,Sell,True,Open,20168.24,0.01,201.6824,0.00,...,20168.24,0.01,201.6824,0.0000,0.020168,-0.020168,-0.01,Open Short,-0.020168,-0.01
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,0,Buy,True,Close,20168.24,0.01,201.6824,19605.10,...,19605.10,0.01,196.0510,5.6314,0.019605,5.611795,0.01,doten Long,5.591627,0.00
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,1,Buy,True,Open,19605.10,0.01,196.0510,0.00,...,19605.10,0.01,196.0510,0.0000,0.019605,-0.019605,0.01,doten Long,5.572022,0.01
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,0,Sell,True,Close,19605.10,0.01,196.0510,19474.00,...,19474.00,0.01,194.7400,-1.3110,0.019474,-1.330474,-0.01,Doten Short,4.241548,0.00
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,1,Sell,True,Open,19474.00,0.01,194.7400,0.00,...,19474.00,0.01,194.7400,0.0000,0.019474,-0.019474,-0.01,Doten Short,4.222074,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0050,1,Buy,True,Open,16949.21,0.01,169.4921,0.00,...,16949.21,0.01,169.4921,0.0000,0.016949,-0.016949,0.01,doten Long,-45.023869,0.01
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,0,Sell,True,Close,16949.21,0.01,169.4921,16968.66,...,16968.66,0.01,169.6866,0.1945,0.016969,0.177531,-0.01,Doten Short,-44.846338,0.00
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,1,Sell,True,Open,16968.66,0.01,169.6866,0.00,...,16968.66,0.01,169.6866,0.0000,0.016969,-0.016969,-0.01,Doten Short,-44.863306,-0.01


In [ ]:
df['total_profit'].sum()

In [ ]:
r


In [ ]:
df = result_to_df(r)
df

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time